# Impact Indicators: the probability of fire in natural vegetation Land Use Land Cover (LULC) classes

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regionmask
import xarray as xr
from c3s_eqc_automatic_quality_control import download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Set paremeters

In [ ]:
# Region of interest
lon_slice = slice(-10, -6)
lat_slice = slice(43, 36)

# Shapefile with regions
shapefile_url = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/shp/NUTS_RG_20M_2021_4326.shp.zip"

# Define forest codes
forest_codes = [50, 60, 61, 62, 70, 71, 72, 80, 81, 82, 90, 100, 160, 170]

## Define request

In [ ]:
# Define request
request = (
    "satellite-fire-burned-area",
    {
        "format": "zip",
        "origin": "esa_cci",
        "sensor": "modis",
        "version": "5_1_1cds",
        "region": "europe",
        "year": ["2016", "2017", "2018", "2019"],
        "variable": "pixel_variables",
        "month": "06",
        "nominal_day": "01",
    },
)

## Download and transform

In [ ]:
# Download and regionalize
ds = download.download_and_transform(
    *request,
    transform_func=utils.regionalise,
    transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": lat_slice},
    chunks={"year": 1},
)
ds = ds.rio.write_crs(4326)
ds["time"] = (ds["time"] + pd.Timedelta("1D")).dt.strftime("%Y-%m")

## Compute cell area

In [ ]:
scaling_factor = 0.002778
resolution_km = scaling_factor * 111.195
indexes = xr.DataArray(range(ds.sizes["latitude"]), dims="latitude")
lats = ds["latitude"].max() - indexes * scaling_factor
grid_cell_area = resolution_km**2 * np.cos(np.radians(lats))
grid_cell_area.attrs = {
    "standard_name": "cell_area",
    "long_name": "Area",
    "units": "km2",
}
ds = ds.assign_coords(cell_area=grid_cell_area)

## Mask regions

In [ ]:
gdf = gpd.read_file(shapefile_url)
gdf = gdf[(gdf["CNTR_CODE"] == "PT") & (gdf["LEVL_CODE"] == 2)]
gdf = gdf[:-2]
gdf = gdf.to_crs(epsg=4326)

regions = regionmask.from_geopandas(gdf, names="NUTS_NAME")
mask = regions.mask_3D(ds)
ds_region = ds.where(mask)
ds_region = (
    ds_region.assign_coords(region=ds_region["names"])
    .drop_vars("names")
    .sortby("region")
)

## Show burned area statistics

In [ ]:
burned_pixels = (
    ds_region["cell_area"].where(ds_region["JD"] > 0).sum(["latitude", "longitude"])
)
burned_pixels.to_pandas()

## Plot maps

In [ ]:
for time, ds_time in ds.groupby("time"):
    ds_time = ds_time.squeeze()
    fig, axs = plt.subplots(
        nrows=1,
        ncols=2,
        figsize=(15, 8),
        subplot_kw={"projection": ccrs.PlateCarree()},
    )

    da = ds_time["JD"] > 0
    da.attrs = {"long_name": "Burned Pixels", "units": "1 = Burned, 0 = Not Burned"}
    plot.projected_map(da, ax=axs[0], cmap="Reds", show_stats=False)
    plot.projected_map(ds_time["CL"], ax=axs[1], cmap="Blues", show_stats=False)

    for ax in axs:
        ax.set_title(time)
        gdf.boundary.plot(ax=ax, color="k", transform=ccrs.PlateCarree())
    plt.show()

## Plot bars

In [ ]:
forest_pixels = (
    ds_region["cell_area"]
    .where(ds_region["LC"].isin(forest_codes))
    .sum(["latitude", "longitude"])
)
da = xr.merge(
    [
        burned_pixels.rename("Total Burned Area"),
        forest_pixels.rename("Forest Burned Area"),
    ]
)
df = da.to_dataframe()[["Total Burned Area", "Forest Burned Area"]]


ylim = [0, 100 * (np.ceil(df.max().max() / 100))]
for region, df_region in df.reset_index().groupby("region"):
    df_region.plot.bar(
        x="time",
        grid=True,
        ylim=ylim,
        title=region,
        color=["orange", "green"],
        ylabel="Burned Area (km²)",
        xlabel="Time",
    )